In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
from pprint import pprint
import platform
print(f'Python version: {platform.python_version()}')
import plotly.plotly as py
import networkx as nx
print (nx.__version__)
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot, plot
import plotly.graph_objs as go

SIZE = 500

init_notebook_mode(connected=True)

Python version: 3.7.0
2.1


In [2]:
tree = ET.parse('./Investment Adviser Representatives Report/IA_Indvl_Feeds20.xml')
root = tree.getroot()

In [3]:
info = []
drp = []
CrntEmp_list = []
Exms_list = []
EmpHs_list = []
for Indvl in root.find('Indvls'):
    #print(Indvl.find('Info').attrib)
    info.append(Indvl.find('Info').attrib)
    index = 0
    if Indvl.find('DRPs'):
        for DRP in Indvl.find('DRPs'):
            #drp.append(DRPs.find('DRP').attrib)
            drp.append(DRP.attrib)
    else:
        drp.append({'hasRegAction': np.nan,
                      'hasCriminal': np.nan,
                      'hasBankrupt': np.nan,
                      'hasCivilJudc': np.nan,
                      'hasBond': np.nan,
                      'hasJudgment': np.nan,
                      'hasInvstgn': np.nan,
                      'hasCustComp': np.nan,
                      'hasTermination': np.nan})
    for CrntEmps in Indvl.findall('CrntEmps'):
        temp = []
        for CrntEmp in CrntEmps:
            temp.append(CrntEmp.attrib)
        CrntEmp_list.append(temp)
    for Exms in Indvl.findall('Exms'):
        temp = []
        for Exm in Exms:
            temp.append(Exm.attrib)
        Exms_list.append(temp)
    for EmpHss in Indvl.findall('EmpHss'):
        temp = []
        for EmpHs in EmpHss:
            temp.append(EmpHs.attrib)
        EmpHs_list.append(temp)
        
# Exms_list

In [4]:
CrntEmp_df = pd.DataFrame(CrntEmp_list)
mapper={}
for i in list(CrntEmp_df.columns):
    mapper[i] = 'CrntEmp_{}'.format(i)
CrntEmp_df.rename(columns=mapper,inplace=True)
# CrntEmp_df

Exms_df = pd.DataFrame(Exms_list)
mapper={}
for i in list(Exms_df.columns):
    mapper[i] = 'Exm_{}'.format(i)
Exms_df.rename(columns=mapper,inplace=True)
# Exms_df

EmpHss_df = pd.DataFrame(EmpHs_list)
mapper={}
for i in list(EmpHss_df.columns):
    mapper[i] = 'EmpHss_{}'.format(i)
EmpHss_df.rename(columns=mapper,inplace=True)

data = pd.concat([pd.DataFrame(info),pd.DataFrame(drp),CrntEmp_df,Exms_df,EmpHss_df],axis=1)
print(data.head())
print(data.columns.values)

  actvAGReg  firstNm  indvlPK    lastNm  \
0         Y     PAUL  1559874  WEINBERG   
1         Y     JOHN  1496345    MACCHI   
2         N  ZACHARY  1587693    SLAGEL   
3         Y   STEVEN  1401631  BISCHOFF   
4         Y  TIMOTHY  1520961   SHEERIN   

                                                link    midNm sufNm  \
0  http://www.adviserinfo.sec.gov/IAPD/Individual...      NaN   NaN   
1  http://www.adviserinfo.sec.gov/IAPD/Individual...   LEONTY   NaN   
2  http://www.adviserinfo.sec.gov/IAPD/Individual...   TANNER   NaN   
3  http://www.adviserinfo.sec.gov/IAPD/Individual...  WALLACE   NaN   
4  http://www.adviserinfo.sec.gov/IAPD/Individual...     MARK   NaN   

  hasBankrupt hasBond hasCivilJudc    ...    EmpHss_16 EmpHss_17 EmpHss_18  \
0         NaN     NaN          NaN    ...         None      None      None   
1         NaN     NaN          NaN    ...         None      None      None   
2           N       N            N    ...         None      None      None   
3 

In [5]:
%%time
#make graph

data = data.head(SIZE)
data = data.replace(['Y','N'], [1, 0])
# data['score'] = -data[['hasBankrupt','hasCriminal','hasInvstgn','hasRegAction','hasTermination','hasCustComp']].sum(axis=1)
data['score'] = -data.loc[:,'hasBankrupt':'hasTermination'].sum(axis=1)
data['num_jobs'] = data.count(axis=1)

G=nx.Graph()
G.add_nodes_from(range(data.shape[0]))

emp_data = data.loc[:,'EmpHss_1':'EmpHss_25']

# index=data.indvlPK
# print(index.head())
# adj_mat = pd.DataFrame(index=index, columns=index)
# adj_mat = adj_mat.fillna(0) # with 0s rather than NaNs
# print(adj_mat.shape)

row = emp_data.head(1)
print(emp_data['EmpHss_1'][0])

emp_set_list = []

def add_set(row):
    emp_set = set()
    for emp in row:
        if emp:
            emp_set.add(emp['orgNm'])
    emp_set_list.append(emp_set)

# get sets of employers for every person
emp_data.apply(lambda x: add_set(x), axis=1)

print(len(emp_set_list))
print(len(emp_data))
for emp_set in emp_set_list[:5]:
#     print(len(emp_set))
    print(emp_set)

{'fromDt': '04/2009', 'toDt': '08/2013', 'orgNm': 'BNY MELLON DISTRIBUTORS INC.', 'city': 'KING OF PRUSSIA', 'state': 'PA'}
500
500
{'BNY MELLON DISTRIBUTORS INC.', 'UNEMPLOYED', 'REINHART PARTNERS, INC.', 'OPPENHEIMER & CO. INC.'}
{'MORGAN STANLEY SMITH BARNEY', 'CITIGROUP GLOBAL MARKETS INC.'}
set()
{'EZY LIFT AQUISITION LLC'}
{'WELLS FARGO CLEARING SERVICES, LLC', 'WELLS FARGO ADVISORS LLC'}
Wall time: 33.9 ms


In [6]:
# %%time
from datetime import datetime
from math import log
# get intersections between employers and add as edge
# get date range overlap

employers = []
shared_days = []

# return dates
def find_dates(i, company):
#     print(emp_data.iloc[i].head())
    for emp in emp_data.iloc[i]:
        if emp and emp['orgNm'] == company:
            if 'toDt' in emp:    
                return (datetime.strptime(emp['fromDt'], '%m/%Y'), datetime.strptime(emp['toDt'], '%m/%Y'))
            else:
                return (datetime.strptime(emp['fromDt'], '%m/%Y'), datetime.now())
    print("no match")

for i, emp_set in enumerate(emp_set_list):
    for j, emp_set_2 in enumerate(emp_set_list[i+1:], i+1):
        intersect = emp_set & emp_set_2
        if (len(intersect) > 0):
#             print(emp_set, emp_set_2)
#             print(intersect)
            
            for company in intersect:
                dates = find_dates(i, company)
                dates_2 = find_dates(j, company)
                assert dates, "no 1"
                assert dates_2, "no 2"
                
                latest_start = max(dates[0], dates_2[0])
                earliest_end = min(dates[1], dates_2[1])
                months = (earliest_end - latest_start).days//30 + 1

                if months > 0:
#                     delta = log(delta)
                    delta = 1
                    G.add_edge(i,j,weight=delta)
                    employers.append(list(intersect))
                    shared_days.append(months)

In [7]:
%%time
#remove isolates

print(len(list(nx.isolates(G))))
# G.remove_nodes_from(list(nx.isolates(G)))

220
Wall time: 0 ns


In [8]:
%%time
pos=nx.spring_layout(G, weight='weight')
# pos=nx.fruchterman_reingold_layout(G)

print(len(pos))

500
Wall time: 2.51 s


In [9]:
# labels=['Bank-1', 'Property','Assets', 'Bank-2', 'Offshore', 'Paradise-papers', 'Glencore', 'Hoifu-Energy', 'Bank-3']

Xn=[pos[k][0] for k in range(len(pos))]
Yn=[pos[k][1] for k in range(len(pos))]
# Xn=[pos[k][0] for k in range(400)]
# Yn=[pos[k][1] for k in range(400)]

Xe=[]
Ye=[]
Xm=[]
Ym=[]

print([e for e in G.edges()][:10])

for e in G.edges():
    Xe.extend([pos[e[0]][0], pos[e[1]][0], None])
    Ye.extend([pos[e[0]][1], pos[e[1]][1], None])
    Xm.append((pos[e[0]][0] + pos[e[1]][0])/2)
    Ym.append((pos[e[0]][1] + pos[e[1]][1])/2)
    
print(Xe[:10])

#get names, links
names = data.firstNm.map(str) + ' ' + data.midNm.map(str) + ' ' + data.lastNm.map(str) + '<br>' + data.link.map(str) + '<br>' + data.num_jobs.map(str)

middle_node_trace = go.Scatter(
    x=Xm,
    y=Ym,
    text=list(zip(employers, shared_days)),
    mode='markers',
    hoverinfo='text',
    marker=go.Marker(
        opacity=0
    )
)    
    
trace_edges=dict(type='scatter',
         mode='lines',
         x=Xe,
         y=Ye,
         line=dict(width=1, color='#888'),
        )

trace_nodes = go.Scatter(
    x=Xn,
    y=Yn,
    text=names,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Bluered',
        reversescale=True,
        color=data.score,
        size=12,
#         size=[],
        colorbar=dict(
            thickness=15,
            title='Reputation',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

# trace_nodes['marker']['size'] = []
# for node, adjacencies in enumerate(G.adjacency()):
#     trace_nodes['marker']['size'].append(len(adjacencies[1]))
#     trace_nodes['marker']['color']+=tuple([len(adjacencies[1])])
#     print(tuple([len(adjacencies[1])]))
#     node_info = '# of connections: '+str(len(adjacencies[1]))
#     trace_nodes['text']+=tuple([node_info])

[(0, 200), (1, 5), (1, 10), (1, 15), (1, 21), (1, 22), (1, 23), (1, 36), (1, 38), (1, 69)]
[0.1510113626718521, 0.09501438587903976, None, -0.014087404124438763, -0.009931562468409538, None, -0.014087404124438763, 0.0025473160203546286, None, -0.014087404124438763]


C:\Users\pazzu\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.




In [10]:


fig = go.Figure(data=[trace_edges, trace_nodes, middle_node_trace],
# fig = go.Figure(data=[trace_edges, trace_nodes],
             layout=go.Layout(
                title='<br>Network graph of Investment Advisors',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Nodes are individuals, edges are concurrent employment weighted by time. Reputation score is calculated as a sum of positive and negative features.",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

# py.iplot(fig, filename='networkx')
iplot(fig, filename='networkx')
# fig = dict(data=[trace_edges, trace_nodes], layout=layout)
# iplot(fig)
